In [ ]:
import pandas as pd
import numpy as np
import math

from scipy.stats import spearmanr

from sklearn.neighbors import NearestNeighbors
from sklearn.utils import resample
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE

from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

from yellowbrick.cluster import KElbowVisualizer

import p1_functions

# Set random seed
np.random.seed(42)

# 1 Loading the Data, Preprocessing, Initial Data Analysis

In [ ]:
df = pd.read_csv(r"https://archive.ics.uci.edu/ml/machine-learning-databases/00292/Wholesale%20customers%20data.csv")
df_og = df.copy()
df

In [ ]:
# Correct the typo
df.rename(columns = {'Delicassen':'Delicatessen'}, inplace = True)

#Describe the dataframe
df.describe()

# Number of distinct elements per column
#df.nunique()

# Check for NaNs
#df.isnull().values.any()

#### Attribute Information:
CHANNEL: customers' Channel - Horeca (Hotel/Restaurant/Cafe) or Retail channel (Nominal)
REGION: customers' Region - Lisnon, Oporto or Other (Nominal)
FRESH: annual spending (m.u.) on fresh products (Continuous)
MILK: annual spending (m.u.) on milk products (Continuous)
GROCERY: annual spending (m.u.)on grocery products (Continuous)
FROZEN: annual spending (m.u.)on frozen products (Continuous)
DETERGENTS_PAPER: annual spending (m.u.) on detergents and paper products (Continuous)
DELICATESSEN: annual spending (m.u.)on and delicatessen products (Continuous)

In [ ]:
# Get nominal and numerical attributes
dtype = df.dtypes
cat_features = ["Channel", "Region"] #df[["Channel", "Region"]]
num_features = df.drop(columns=["Channel", "Region"]).columns.tolist()

print("Nominal features:", cat_features)
print("Numerical features:", num_features)

### Basic statistical visualisations of the data

In [ ]:
# Plot distributions of attributes: NOMINAL (barplots)
p1_functions.cat_plot(df, cat_features)

In [ ]:
# Plot overall spending per category
fig = plt.figure(figsize=(5,5))
df[num_features].sum().plot.bar()
plt.title("Spending per Category")
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
# Plot distributions of attributes: NUMERICAL (histograms, boxplot)
# Plot correlations (pairwise scatterplots and heatmap)
p1_functions.num_plot(df[num_features])

#### Observation: The distributions are heavy tailed -> apply the log function to the continuous features so that the distribution becomes compressed for large values and expanded for small values

In [ ]:
# Drop meta-data: Channel and the Region indicators
df = df.drop(columns=cat_features)

# Save the original dat set without meta-data
df_og = df.copy()
df_overview = df.copy() # we will use this to append the scores for an overview. Calculations are performed on df

# x ← log(x + 1)
df = np.log(df + 1)

### Recomputed plots

In [ ]:
# Recompute plots after the transformation
p1_functions.num_plot(df)

#### Observation: The log transformation produces more normally distributed attributes, however the statistics can not be reliably computed (e. g. correlations). Also, some lower values can now be considered extreme, while very few higher values still possess this quality (see boxplots).

# 2 Detecting Anomalies

## Hard-Min Score

Compute the distance to the nearest neighbor for each instance. This measure is used as an outlier score. We assume that this measure is not very biased, but has a high variance. We therefore assume that using a bootstrap with the Hard-Min approach produces a more accurate and robust outlierness score. Later on, (together with some visual inspections) we evalute the accuracy of a more robust, but more biased estimator of the scores. Note that we are interested in the resulting ranking of the scores and not the scores per se.


### Fake it till you make it: Creating an artificial "Ground Truth"

In [ ]:
# Setting the HYPERPARAMETERS
N_BOOTSTRAP = 1000
SAMPLE_SIZE_FRAC  = 0.5
OUTLIERS_FRAC = 0.04

The Spearman correlation as well as the scores increase with a higher sample size fraction (nearest neighbor distances increase monotonically since we consider fewer data points). To assess the bias of the soft Estimator, which we will consider later for different values of gamma, we classify the "OUTLIERS_FRAC" % of the most likely outliers from the hard-min bootstrap and check the accuracy.

The Sample size fraction is chosen in a way s.t. the spearman correlation with the Hard-Min on the whole dataset is high, but not too high.

The outliers frac of 4 % results in 14 samples from 440 in the dataset which seems a reasonable guess. This parameter can be adjusted given further domain knowledge or if an indicator is needed to make the outlier scores even more robust.

In [ ]:
# Create Samples for Bootstrap approach
samples = [resample(df, n_samples=math.ceil(len(df) * SAMPLE_SIZE_FRAC), replace=False) for i in range(N_BOOTSTRAP)]

def hardmin_score(df):
    nbrs = NearestNeighbors(n_neighbors=2, algorithm="ball_tree").fit(df)
    distances, indices = nbrs.kneighbors(df)
    outlier_score_min = np.square(distances[:, 1])
    return outlier_score_min


def hardmin_bootstrap(n_bootstrap=N_BOOTSTRAP, sample_size_frac=SAMPLE_SIZE_FRAC, data=df, samples = samples):
        hard_min_scores_bootstrap = pd.DataFrame({i: np.full(N_BOOTSTRAP, np.nan) for i in range(len(df))})
        #samples = [resample(df, n_samples=math.ceil(len(df) * SAMPLE_SIZE_FRAC), replace=False) for i in range(N_BOOTSTRAP)]
        for i in range(0, N_BOOTSTRAP):
                sample = samples[i]
                hard_min_scores_bootstrap.loc[i, sample.index] = hardmin_score(sample)
                
        return hard_min_scores_bootstrap

Example for the bootstrap values table below:

In [ ]:
hard_min_scores_bootstrap = hardmin_bootstrap(df)
display(hard_min_scores_bootstrap)

We save the Hard-Min outlier scores for the whole dataset as well as the mean outlier scores over the bootstrap samples without replacement. We than compare the two.

In [ ]:
df_overview["hardmin_score"] = hardmin_score(df)
df_overview["hardmin_bootstrap_score"] = hardmin_bootstrap(data=df).mean()
#display(df_overview.head())
df_overview.sort_values(by="hardmin_bootstrap_score", ascending=False).head(10)

In [ ]:
# Check difference! Stabilized it a little bit
sns.boxplot(np.abs(df_overview["hardmin_bootstrap_score"] - df_overview["hardmin_score"])).set_title("Difference between hardmin on whole data set vs. mean on bootstrap")
plt.show()

### Everything gets down to choosing the right evaluation metric

Compare Hard-Min and Hard-Min-bootstrap appraoches by statistical means as well as "artificial" outlier classification.

Later when we will try to choose an optimal gamma, we will use the same approach!

In [ ]:
def score_bias_eval(scores, baseline=df_overview["hardmin_bootstrap_score"].to_numpy(), outlier_frac=OUTLIERS_FRAC,
                    return_p_val=False):
    n_outliers = math.ceil(len(baseline) * outlier_frac)
    spr = spearmanr(scores, baseline)
    hardmin_bootstrap_ranking = np.argsort(baseline * (-1))
    hardmin_outliers = hardmin_bootstrap_ranking[:n_outliers]
    score_outlier_ranking = np.argsort(scores * (-1))
    score_outliers = score_outlier_ranking[:n_outliers]

    accuracy = len(set(hardmin_outliers).intersection(score_outliers)) / n_outliers

    return spr[0], accuracy

spr, acc = score_bias_eval(scores=df_overview["hardmin_score"])

print(f"Bootstrap Accuracy: {acc:.2}% \nSpearman results: {spr}")

## The Soft-Min Score/KDE approach

To get a better understanding what we'll do next, we will compute the soft-min scores for a fixed Gamma value and compare them to our baseline/ground-truth.

In [ ]:
GAMMA = 1

In [ ]:
nbrs = NearestNeighbors(n_neighbors=len(df), algorithm="ball_tree").fit(df)
distances, indices = nbrs.kneighbors(df)
distances = np.square(distances)

def softmin(z, gamma=GAMMA):
    return (-1 / gamma) * np.log( (1 / (len(z) - 1)) * np.sum(np.exp(-gamma * z)) )


df_overview["outlier_score_softmin"] = np.apply_along_axis(softmin, 1, distances[:, 1:], GAMMA)
spr, acc = score_bias_eval(scores=df_overview["outlier_score_softmin"])

print(f"Bootstrap Accuracy: {acc:.2}% \nSpearman results: {spr}")

In [ ]:
# Visually compare distributions of the scores
fig, axs = plt.subplots(2,2, figsize=(10,10))

sns.boxplot(df_overview["outlier_score_softmin"], ax=axs[0, 0])
sns.histplot(df_overview["outlier_score_softmin"], ax=axs[1, 0])
sns.boxplot(df_overview["hardmin_bootstrap_score"], ax=axs[0, 1])
sns.histplot(df_overview["hardmin_bootstrap_score"], ax=axs[1, 1])
plt.show()

We can see that the distributions are quite similar although they "operate" on different scales. When we thought about how to evaluate different measures, first we tried to use the variance of the outlier scores as a possible measure for the signal power of the respective score. But the Soft-Min function operates on different scales for different values of gamma which makes a comparison hard. Just because a different scale/variance is used, this does not have to implicate that the score is better or worse. Instead, we are interested in the resulting ranking. That's why we will focus on the spearman rank correlation as well as our "artificial" accuracy specified above in the Hard-Min bootstrap case.

Note that we tried different approaches, e.g. leaving the factor of 1/gamma out of the Soft-Min function to get comparable scales. We will also introduce some visual inspections. For example, in the following we can see how the Soft-Min score for a given instance changes for different values of gamma. We will use this for each instance to evaluate the development of the scores.

In [ ]:
# 338 has had a high outlier score
instance = 338
gammas = np.linspace(0.1, 2, 100)
g = sns.lineplot(
    x=gammas, y=[softmin(distances[instance, 1:], gamma) for gamma in gammas]
)
g.set_xlabel("gamma")
g.set_ylabel("softmin") 
plt.show()

We can clearly see how the scale of the Soft-Min score changes!

## Bootstrap/Robustness evaluation & the curse of choosing a bandwidth

Since the Soft-Min score can be interpreted as a KDE-approach, the chosen gamma corresponds to the inverse of the bandwidth or variance of the used Gaussian distributions. Because of that, a high gamma leads to less biased estimates but with the cost of introducing a higher estimator variance! It is known that finding an appropriate bandwidth is a crucial and non-trivial task (and also way more important than choosing a kernel).

Consequently, if we want to choose gamma, we have to evaluate the bias and variance of our estimator. To do this, we will use the bootstrap approach without resampling on a fraction size of the whole dataset. This will be set as a Hyperparameter and has to be chosen in a way that the data is varying enough but still carries enough information/is representative for the whole dataset. To evaluate the model variance for a given gamma, we will compute the score for each instance for each bootstrap round, then compute the within-variance of each sample across the different bootstrap rounds and finally aggregate the variances for each sample with a mean across the whole dataset.

To analyse the bias, we will use the approaches discussed above.

In [ ]:
def softmin(z, gamma):
    return - (1/gamma)  * np.log(1 / (len(z) - 1) * np.sum(np.exp(-gamma * z)))

In [ ]:
N_BOOTSTRAP = 100
gamma_range = np.linspace(0.1, 2, 20)

scores = np.full((len(df), N_BOOTSTRAP, len(gamma_range)), np.nan) # customer x bootstrap round x gamma

for i in range(0, N_BOOTSTRAP):

    sample = samples[i]

    nbrs = NearestNeighbors(n_neighbors=len(sample), algorithm="ball_tree").fit(sample)
    distances, indices = nbrs.kneighbors(sample)
    distances = np.square(distances[:, 1:])
    
    for j, gamma in enumerate(gamma_range):
        scores[sample.index, i, j] = np.apply_along_axis(softmin, 1, distances, gamma)

In [ ]:
# verify normal distribution arround N_BOOTSTRAP * frac
# sns.histplot(np.sum(np.isnan(scores) == False, axis=1)[:,0]).set_xlabel("# not sampled")
# plt.show()

In [ ]:
np.nanmean(scores, axis=1).shape

In [ ]:
# average outlier score per sample, per gamma
avg_score = np.nanmean(scores, axis=1)

display(avg_score.shape)
print("average outlier score of instance 0:")
display(avg_score[0])

print("average outlier score of instance 338:")
display(avg_score[338])



fig = plt.figure(figsize=(10,7))
for i in range(len(scores)):
    plt.plot(gamma_range, avg_score[i], linewidth=0.3)
    plt.xlabel("Gamma")
    plt.ylabel("Soft-Min Score (mean)")
    plt.title(f"Mean Soft-min Scores over {N_BOOTSTRAP} bootstraps for each customer")
plt.show()

While the Soft-Min Scores are getting lower for all the samples for increasing gammas, the ranking more or less stays the same! Especially the ones with the highest scores are still just as separable. Hence, we discarded the variance over the scores as a potential measure of bias.

In [ ]:
# calculate the spread over bootstrap dimension aka within sample variance
spread = np.nanvar(scores, axis=1)

# Comparing inlier vs outlier
display(spread.shape)
print("Vars of instance 0:")
display(spread[0])

print("Vars of instance 338:")
display(spread[338])


# all spreads per instances
fig = plt.figure(figsize=(10,7))
for i in range(len(scores)):
    plt.plot(gamma_range, spread[i], linewidth=0.3)
    plt.xlabel("Gamma")
    plt.ylabel("Var(Soft-Min Score)")
    plt.title(f"Variance of Soft-Min Scores over {N_BOOTSTRAP} bootstraps for each customer")
plt.show()


We can see that the variance of the model increases with increasing gamma just as expected. What strikes out is that it increases way more for outliers than for inliers. Since our purpose is to detect outliers, we have to carefully choose our gamma value to have robust estimates.

In [ ]:
# Check mean behaviour as well
g = sns.scatterplot(x=gamma_range, y=np.mean(np.nanvar(scores, axis=1), axis=0))
g.set_xlabel('Gamma')
g.set_ylabel('Mean(Var(Soft-Min Score))')
plt.show()

Although the variance is way higher for the outliers, the mean reflects the model variance on a low scale as well.

In [ ]:
#print(avg_score.shape)
spr, acc = score_bias_eval(scores=avg_score[:, 1])
print(f"Bootstrap Accuracy: {acc:.2}% \nSpearman results: {spr}")

In [ ]:
def mean_var_of_most_anomalous(customer_variances, n_most_outliers_considerd = OUTLIERS_FRAC):
    n_most_outliers_considerd = math.ceil(len(customer_variances) * OUTLIERS_FRAC)
    var_of_outliers = np.sort(customer_variances)[::-1][:n_most_outliers_considerd]
    return np.mean(var_of_outliers)


In [ ]:
bias_measures = np.apply_along_axis(score_bias_eval, 0, avg_score)
var_measures = np.apply_along_axis(mean_var_of_most_anomalous, 0, spread)
gamma_overview = pd.DataFrame(bias_measures, index=["Spearman Corr", "Accuracy"])
gamma_overview.loc["Model Var", :] = var_measures
gamma_overview

In [ ]:
# TODO Requires a motivation of choosing just the outlier points for computing model variance (relevant for the next section)

g = sns.scatterplot(x=gamma_range, y=var_measures)
g.set_xlabel('Gamma')
g.set_ylabel('Model Variance')
plt.show()

## Choosing the right gamma

In [ ]:
figure, axs = plt.subplots(1,2, figsize=(15, 5))

g = sns.scatterplot(y=gamma_overview.loc["Spearman Corr", ], x=gamma_overview.loc["Model Var", ], hue=gamma_range, ax=axs[0])
g.set_xlabel('Model Variance')
g.set_ylabel('Spearman Correlation')

g = sns.scatterplot(y=gamma_overview.loc["Accuracy", ], x=gamma_overview.loc["Model Var", ], hue=gamma_range, ax=axs[1])
g.set_xlabel('Model Variance')
g.set_ylabel('Accuracy')

plt.show()

We can clearly see that with increasing gamma, the model robustness as well its accuracy decreases. We have to choose a gamma which achieves a fair Trade-Off.

Arguably, the gamma value should be at least above 0.3. Also, a Spearman Correlation of .9 is already quite high while the curve is getting very steep when the Spearman Correlation increases. The corresponding gamma value should therefore not exceed 1.2. For further analysis we take the highest value which has a relationship with model varoiance that could be approximated by a line.

This is leeds to the Gamma value of 1.


One could also restrict the spearman correlation on the most probable outliers just like in the case of the model variance. But since we also use the "artificial" accuracy, we can see that the Score is apropiate even in the "crucial" area of interest.

# 3 Explaining Anomalies

In [ ]:
def relevance(x, yj, gamma):
    """Calculate layer-wise relevance propagation
    @x: array of instances
    @yj: softmin scores of all j instances
    @gamma: gamma used for softmin 
    """

    Rji = np.zeros(x.shape)

    # calculation per instance
    for j in range(len(x)):

        # mask is used to exclude the current instance j
        mask = np.full((len(x)), True)
        mask[j] = False

        # save xk - xj
        xk_j = x - x[j]

        # calculate zk = ||xj - xk||^2
        zk = np.square(np.linalg.norm(xk_j, axis=1))

        # 1. First, one identifies to what extent each data point has contributed to the anomaly score of instance j
        temp = np.exp(-gamma * zk[mask])
        Rk = temp / np.sum(temp) * yj[j]

        # 2. Then, these scores can be propagated back to the input features by observing that the (squared)
        # Euclidean distance entering the anomaly score can be decomposed in terms of individual components:
        Rji[j, :] = np.sum(np.square(xk_j)[mask] / zk[mask][:, None] * Rk[:, None], axis=0)

    return Rji


In [ ]:
gamma = 1

# Calculate anomaly scores
nbrs = NearestNeighbors(n_neighbors=len(df), algorithm="ball_tree").fit(df)
distances, _ = nbrs.kneighbors(df)
yj = np.apply_along_axis(softmin_og, 1, np.square(distances[:, 1:]), gamma)

x = df.to_numpy()
Rji = relevance(x, yj, gamma)

# confirm conservation property
np.all(np.sum(Rji, axis=1) - yj <= 1e-14)

In [ ]:
X_embedded = TSNE(n_components=2, learning_rate='auto', init='pca', perplexity=30).fit_transform(df)

f, ax = plt.subplots(figsize=(10,8))
points = ax.scatter(x=X_embedded[:,1], y=X_embedded[:,0], c=yj, s=30, cmap=plt.cm.get_cmap('viridis_r'))
f.colorbar(points)
ax.set_ylabel("t-SNE 1")
ax.set_xlabel("t-SNE 2")
ax.set_title("Outlier score in 2D projection (t-SNE) ")
plt.show()


In [ ]:
instance = 338
g = sns.barplot(x=Rji[instance], y=list(map(str, list(zip(df_og.iloc[instance].index, df_og.iloc[instance].values)))))
g.set_xlim(0,np.max(Rji))
plt.show()

# TODO WHY IS Fresh = 3 not relevant for outlier score?
print("Ri:", Rji[instance])
print("Outlier score:", yj[instance])
display(df.iloc[instance])
display(df.describe())

## Experiment to use explanations of these outlier scores for the reproducibility experiment in task 2

In [ ]:
# 50% random sample without replacement
# for each gamma:
#   calculate anomaly scores for sample instances
#   calculate relevance 
#   save relevance (per component) for later analysis

gamma = 1

N_BOOTSTRAP = 100

Rji_BS = np.full((440, N_BOOTSTRAP, len(gamma_range), 6), np.nan)

for i in range(N_BOOTSTRAP):

    sample = df.sample(frac=0.5)

    nbrs = NearestNeighbors(n_neighbors=len(sample), algorithm="ball_tree").fit(sample)
    distances, _ = nbrs.kneighbors(sample)
    distances = np.square(distances[:, 1:])

    for k, g in enumerate(gamma_range):
        yj = np.apply_along_axis(softmin_og, 1, distances, g)
        Rji_BS[sample.index, i, k, :] = relevance(sample.to_numpy(), yj, g)


In [ ]:
# for each instance, for each sample, for each gamma, the relevance per component
Rji_BS.shape

### Experiment for all components at once

In [ ]:
# analyse the mean Euclidean distance between the relevance components over all bootstrap samples (per gamma and per instance)
# since the range of the anomaly scores differes with gamma, the range of the relevance components will also depend on gamma

from scipy.spatial import distance

mean_dist = np.zeros((len(Rji_BS), len(gamma_range)))
var_dist = np.zeros((len(Rji_BS), len(gamma_range)))

# iterate over instances
for j in range(len(Rji_BS)):
    # iterate over gammas
    for k in range(len(gamma_range)):
        mean_dist[j,k] = np.nanmean(distance.pdist(Rji_BS[j,:,k,:]))
        var_dist[j,k] = np.nanvar(distance.pdist(Rji_BS[j,:,k,:]))

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(16, 12))
axs = axs.flatten()


for i in range(len(var_dist)):
    axs[0].plot(gamma_range, var_dist[i], linewidth=0.3)

sns.scatterplot(x=gamma_range, y=np.mean(var_dist, axis=0), ax=axs[2])



for i in range(len(mean_dist)):
    axs[1].plot(gamma_range, mean_dist[i], linewidth=0.3)

sns.scatterplot(x=gamma_range, y=np.var(mean_dist, axis=0), ax=axs[3])


axs[0].set_ylabel("instance variance of component distance")
axs[1].set_ylabel("instance mean of component distance")
axs[2].set_ylabel("mean of instance variance of component distance")
axs[3].set_ylabel("variance of instance mean of component distance")
for ax in axs:
    ax.set_xlabel("gamma")

plt.show()


### Experiment PER COMPONENT

In [ ]:
component = 0

fig, axs = plt.subplots(2, 2, figsize=(16, 12))
axs = axs.flatten()


spread = np.nanvar(Rji_BS[:, :, :, component], axis=1)

for i in range(len(scores)):
    axs[0].plot(gamma_range, spread[i], linewidth=0.3)

sns.scatterplot(x=gamma_range, y=np.mean(spread, axis=0), ax=axs[2])




avg_rel = np.nanmean(Rji_BS[:, :, :, component], axis=1)

for i in range(len(scores)):
    axs[1].plot(gamma_range, avg_rel[i], linewidth=0.3)

sns.scatterplot(x=gamma_range, y=np.var(avg_rel, axis=0), ax=axs[3])


axs[0].set_ylabel("instance variance of relevance")
axs[1].set_ylabel("instance mean of relevance")
axs[2].set_ylabel("mean of instance variance of relevance")
axs[3].set_ylabel("variance of instance mean of relevance")
for ax in axs:
    ax.set_xlabel("gamma")

plt.show()


In [ ]:
# TODO do these findings match with our other findings?
# TODO the components behave differently - why?
# TODO remove outliers!!

# 4 Cluster Analysis

#### Observation:
Since the data does not appear to have natural cluster formations, and DBSCAN algorithm was not finding any bigger clusters, we decided to use K-means clustering algorithm. This was we can partition wholesale cusptmers into groups of relatively similar size that share some tendencies in their spending.

### Selecting K parameter for K-means

Compute visualisation to identify the optimal inflection point for K in [2, 15]:
* elbow (distortion score): the sum of squared distances from each point to its assigned center:
* silhouette score: the mean Silhouette Coefficient of all samples;
* Calinski-Harabasz score: the ratio of dispersion between and within clusters.

In [ ]:
# K-means initialised with greedy k-means++ algorithm, over 100 initialisations
kmeans = KMeans(init="k-means++", n_init=100, random_state=42)

fig = plt.figure(figsize=(20,10))
gs = gridspec.GridSpec(2,3)
ax = {}

for i, metr in zip(range(3), ['distortion', 'silhouette', 'calinski_harabasz']):
    ax[i] = fig.add_subplot(gs[i])
    ax[i] = visualizer = KElbowVisualizer(kmeans, k=(2,16), metric=metr, timings=False, show=True)
    visualizer.fit(df)
    visualizer.finalize()

plt.show()

#### Observation:
There is no clear infliction point in the elbow plot of the distoriotn score, we can assume K = 6 to be a reasonable trade-off point. Similar result is obtained by computing the Calinski-Harabasz index.
The silhouette score is strongly reduced for K > 3, however we assume that more than three clusters is reasonable in order to partition the customers in this application.

#### In addition we perform K-means clustering for K in the range [2, 6] and inspect the silhouette scores for the clusters.

In [ ]:
p1_functions.silhouette_analysis(min_k=2, max_k=8, X=df, Umap=True)

#### Observation:
It seems that the most plausible solution for K > 3 is K = 5 or K = 6. The silhouette scores are overall not very satisfying, but they are a bit higher for these two clusterings. As for individual clusters, the clustering with K = 6 has a small cluster with index 0, which in not having a high score, but seems to allow reorganisation and slightly better scoring for the other clusters. Both these options could be reasonable and would need to be discussed in the context of the application. Since we do not have this context, we fix K to six as this number of clusters also provides a lower distortion score.

In [ ]:
# Fit k-Means and visualise using UMAP and t-SNE
K = 6
kmeans = KMeans(n_clusters=K, init="k-means++", n_init=100, random_state=42).fit(df)
p1_functions.visualise_kmeans(df, kmeans)

In [ ]:
print("k-Means cluster sizes:")
for cluster in set(kmeans.labels_):
    filter = kmeans.labels_ == cluster
    print("Cluster:", cluster, ":", sum(kmeans.labels_ == cluster))

#### Interpretation of the clustering
For an interpretation of the clustering, we plot average spending of cluster members for the different product categories.
We computed means and standard deviations for each category within each cluster, as well as boxplots for each clsuter within each category.

In [ ]:
p1_functions.clusters_stats(df, kmeans)